In [96]:
import numpy as np
import pandas as pd
import pandas_profiling
import pymysql

In [57]:
tripData1 = pd.read_csv('/dltraining/datasets/Trip Data/trip_data_1.csv', nrows = 10000)

In [30]:
tripData1.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,-74.006683,40.731781,-73.994499,40.750660
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,-74.004707,40.737770,-74.009834,40.726002
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,-73.976250,40.748528,-74.002586,40.747868


In [114]:
tripFare1 = pd.read_csv('/dltraining/datasets/Trip Fare/trip_fare_1.csv', nrows = 10000) 

#remove the leading spaces
tripFare1.columns = ['medallion', 'hack_license', 'vendor_id', 'pickup_datetime',
       'payment_type', 'fare_amount', 'surcharge', 'mta_tax',
       'tip_amount', 'tolls_amount', 'total_amount']

In [99]:
tripFare1.head()

,medallion,hack_license,vendor_id,pickup_datetime,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,2013-01-01 15:11:48,CSH,6.5,0.0,0.5,0.0,0.0,7.0
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,2013-01-06 00:18:35,CSH,6.0,0.5,0.5,0.0,0.0,7.0
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,2013-01-05 18:49:41,CSH,5.5,1.0,0.5,0.0,0.0,7.0
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,2013-01-07 23:54:15,CSH,5.0,0.5,0.5,0.0,0.0,6.0
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,2013-01-07 23:25:03,CSH,9.5,0.5,0.5,0.0,0.0,10.5


# Database Connection 

In [117]:
host="nyctaxi.cq3rwooo9ghy.ap-southeast-2.rds.amazonaws.com"
port=3306
dbname="NYCTaxiDB"
user="Administrator"
password="X5LdUuzaZoY7CvUO7Gmpaw4Hed"

conn = pymysql.connect(host, user=user,port=port,
                           passwd=password, db=dbname)
# create object to talk to the db
cursorObject = conn.cursor()

In [118]:
# check to see if there are any tables
sqlQuery = "show tables"
cursorObject.execute(sqlQuery)
for x in cursorObject:
  print(x)

('fareData',)
('tripData',)


In [61]:
sqlQuery = "CREATE TABLE tripData(id INT AUTO_INCREMENT PRIMARY KEY, medallion char(32), hack_license char(32), vendor_id varchar(32), rate_code int, store_and_fwd_flag char(5), pickup_datetime datetime, dropoff_datetime datetime , passenger_count int, trip_time_in_secs int, trip_distance float(10,5), pickup_longitude float(12, 7), pickup_latitude float(12, 7), dropoff_longitude float(12, 7), dropoff_latitude float(12, 7))"
cursorObject.execute(sqlQuery)

0

In [62]:
sqlQuery = "CREATE TABLE fareData(id INT AUTO_INCREMENT PRIMARY KEY, medallion char(32), hack_license char(32), vendor_id varchar(32), pickup_datetime datetime, payment_type varchar(32), fare_amount float(10,3), surcharge float(10,3), mta_tax float(10,3), tip_amount float(10,3), tolls_amount float(10,3), total_amount float(10,3))"
cursorObject.execute(sqlQuery)

0

In [63]:
sqlQuery = "show tables"
cursorObject.execute(sqlQuery)
for x in cursorObject:
  print(x)

('fareData',)
('tripData',)


In [119]:
tripFare1.to_sql(con=conn, name='fareData_1', if_exists='append', index=False)

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': not all arguments converted during string formatting

In [124]:
sqlQuery = "select count(*) from fareData"
cursorObject.execute(sqlQuery)
for x in cursorObject:
  print(x)

(0,)


In [126]:
sqlQuery="ALTER TABLE NYCTaxiDB.fareData AUTO_INCREMENT =1"
cursorObject.execute(sqlQuery)
for x in cursorObject:
  print(x)

In [121]:
sqlQuery = "SELECT name FROM sqlite_master WHERE type='table' AND name='fareData_1'"
cursorObject.execute(sqlQuery)
for x in cursorObject:
  print(x)

ProgrammingError: (1146, "Table 'NYCTaxiDB.sqlite_master' doesn't exist")

In [115]:
pandas_profiling.ProfileReport(tripFare1)

/opt/conda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/opt/conda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/opt/conda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


Number of variables,11
Number of observations,14776615
Total Missing (%),0.0%
Total size in memory,1.2 GiB
Average record size in memory,88.0 B
Numeric,4
Categorical,5
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [127]:
# close the connection
conn.close()